In [1]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes
!pip install nltk==3.7
!pip install -U Flask
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import hashlib

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# client = storage.Client()
# bucket_name = 'ir_assg3_eithan'

# bucket = client.bucket(bucket_name)

# # Get a list of all the blobs in the folder
# blobs = bucket.list_blobs(prefix='indexes/')

# # Iterate over the list of blobs and download each one to a local file
# for blob in blobs:
#     if 'doc_norm_dict.pickle' in blob.name or 'doc_len_dict.pickle' in blob.name:
#         local_file_name = blob.name[blob.name.rfind("/")+1:]
#         blob.download_to_filename(local_file_name)

In [25]:
from flask import Flask, request, jsonify
import collections
import csv
import gzip
import math
import json

!pip install fasttext
import fasttext
import requests

!pip install gensim
# from gensim.models.wrappers import * 

In [5]:
%cd /home/dataproc
from inverted_index_gcp import InvertedIndex
# from inverted_index_gcp import *
# import inverted_index_gcp as iig
from inverted_index_gcp import * 

/home/dataproc


In [37]:

# import importlib
# importlib.reload(iig)

In [6]:
path_anchor = "/home/dataproc/postings_gcp_anchor"
path_body = "/home/dataproc/postings_gcp_body"
path_title = "/home/dataproc/postings_gcp_title"

In [7]:
# Read indexes
body_index = InvertedIndex.read_index('.',"body_index")
anchor_index = InvertedIndex.read_index('.',"anchor_index")
title_index = InvertedIndex.read_index('.',"title_index")

In [8]:
body_index.bin_path = path_body
anchor_index.bin_path = path_anchor
title_index.bin_path = path_title

In [9]:
with open('doc_id_title_dict.pickle','rb') as f:
    doc_title_dict = pickle.load(f)
    
with open('doc_len_dict.pickle','rb') as f:
    doc_len_dict = pickle.load(f)
    
with open('page_views_dict.pkl','rb') as f:
    page_views_dict = pickle.load(f)

with open('page_rank_dict.pickle','rb') as f:
    page_rank_dict = pickle.load(f)

with open('doc_norm_dict.pickle','rb') as f:
    doc_norm_dict = pickle.load(f)

In [ ]:
with open('queries control group.json', 'rt') as f:
    queries = json.load(f)

In [10]:
from contextlib import closing
def read_posting_list(inverted, w, index_type=""):
    with closing(MultiFileReader()) as reader:
        locs = inverted.posting_locs[w]
        b = reader.read(locs, inverted.df[w] * TUPLE_SIZE,index_type)
        posting_list = []
        for i in range(inverted.df[w]):
            doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
            tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
            posting_list.append((doc_id, tf))
        return posting_list

In [14]:
# Reading posting list example:
# lst = read_posting_list(body_index,"texas", path_body)
# print(len(lst))

In [11]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

def tokenize(text):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    tokens = set(tok for tok in tokens if tok not in all_stopwords)
    return tokens



In [15]:
def search_title():
    ''' Returns ALL (not just top 100) search results that contain A QUERY WORD 
        IN THE TITLE of articles, ordered in descending order of the NUMBER OF 
        QUERY WORDS that appear in the title. For example, a document with a 
        title that matches two of the query words will be ranked before a 
        document with a title that matches only one query term. 

        Test this by navigating to the a URL like:
         http://YOUR_SERVER_DOMAIN/search_title?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of ALL (not just top 100) search results, ordered from best to 
        worst where each element is a tuple (wiki_id, title).
    '''

    # query=["hello world"] - > [(wiki_id, title)] sorted decreasing
    # in a way that the title that contains most words of the query

    res = []
#     query = request.args.get('hello world', '')
    query = "LinkedIn"
#     query = "hello"
    if len(query) == 0:
        return jsonify(res)
    # BEGIN SOLUTION

    counter = collections.Counter()
    for token in tokenize(query):
        for docId_tf in title_index.read_posting_list(token):
            doc_id = docId_tf[0]
            title = doc_title_dict[doc_id]
            counter[(doc_id, title)] += 1

    
#     res = [map(lambda tup:tup[0], counter.most_common())] 
    res = list(map(lambda tup:tup[0], counter.most_common()))
    # END SOLUTION
    return res

In [54]:
print(search_title())

[(970755, 'LinkedIn'), (27769500, 'LinkedIn Pulse'), (36070366, '2012 LinkedIn hack'), (41726116, 'LinkedIn Learning'), (50191962, 'Timeline of LinkedIn'), (57147095, 'LinkedIn Top Companies'), (62976368, 'HiQ Labs v. LinkedIn')]


In [15]:
def search_anchor():
    # query=["hello world"] - > [(wiki_id, title)] sorted decreasing
    # in a way that the title that contains most words of the query

    res = []
#     query = request.args.get('hello world', '')
    query = "LinkedIn"
#     query = "hello"
    if len(query) == 0:
        return jsonify(res)
    # BEGIN SOLUTION

    counter_anchor = collections.Counter()
    for token in tokenize(query):
        for docId_tf in anchor_index.read_posting_list(token):
            doc_id = docId_tf[0]
            title = doc_title_dict[doc_id]
            counter_anchor[(doc_id, title)] += 1

    
#     res = [map(lambda tup:tup[0], counter.most_common())] 
    res = list(map(lambda tup:tup[0], counter_anchor.most_common()))
    # END SOLUTION
    return res

In [31]:
# print(search_anchor())

In [16]:
def get_pageview():
    ''' Returns the number of page views that each of the provide wiki articles
        had in August 2021.

        Test this by issuing a POST request to a URL like:
          http://YOUR_SERVER_DOMAIN/get_pageview
        with a json payload of the list of article ids. In python do:
          import requests
          requests.post('http://YOUR_SERVER_DOMAIN/get_pageview', json=[1,5,8])
        As before YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of ints:
          list of page view numbers from August 2021 that correrspond to the 
          provided list article IDs.
    '''

    res = []
#     wiki_ids = request.get_json()
    wiki_ids = [848, 21105323, 23536538, 23797577]
    if len(wiki_ids) == 0:
        return res
    # BEGIN SOLUTION
    
    res = [page_views_dict.get(doc_id, -1) for doc_id in wiki_ids]

    # END SOLUTION
    return res

In [62]:
print(get_pageview())

[84370, 1042, 30483, 38197]


In [34]:
def get_pagerank():
    ''' Returns PageRank values for a list of provided wiki article IDs. 

        Test this by issuing a POST request to a URL like:
          http://YOUR_SERVER_DOMAIN/get_pagerank
        with a json payload of the list of article ids. In python do:
          import requests
          requests.post('http://YOUR_SERVER_DOMAIN/get_pagerank', json=[1,5,8])
        As before YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of floats:
          list of PageRank scores that correrspond to the provided article IDs.
    '''
    res = []
#     wiki_ids = request.get_json()
    wiki_ids = [848, 3434750, 32927]
    if len(wiki_ids) == 0:
        return jsonify(res)
    # BEGIN SOLUTION

    res = [page_rank_dict.get(doc_id, -1) for doc_id in wiki_ids]
    # END SOLUTION
#     return jsonify(res)
    return res

In [35]:
print(get_pagerank())

[50.7020900020747, 9913.728782160773, 5282.081575765278]


In [12]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

N = 6348910

def normalize(tokens):
    counter = Counter(tokens)
    norm = 0
    for value in counter.values():
        norm += value*value
    if norm == 0 :
        return 0
    return 1/(math.sqrt(norm))


def tf_idf_calc (token, doc_id, tf):
    tf =  tf/doc_len_dict[doc_id]
#     idf = math.log(N/app.body_index.df[token], 2)
    idf = math.log(N/body_index.df[token], 2)
    return tf*idf

def tokenize(text):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    tokens = [tok for tok in tokens if tok not in all_stopwords]
    return tokens

In [18]:
def search_body():
    ''' Returns up to a 100 search results for the query using TFIDF AND COSINE
        SIMILARITY OF THE BODY OF ARTICLES ONLY. DO NOT use stemming. DO USE the 
        staff-provided tokenizer from Assignment 3 (GCP part) to do the 
        tokenization and remove stopwords. 

        To issue a query navigate to a URL like:
         http://YOUR_SERVER_DOMAIN/search_body?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of up to 100 search results, ordered from best to worst where each 
        element is a tuple (wiki_id, title).
    '''
    res = []
#     query = request.args.get('query', '')
    query = 'linkedIn'
    if len(query) == 0:
        return jsonify(res)
    # BEGIN SOLUTION

    similarity_dict = {}
    tokens = tokenize(query)
    for token in tokens:
        for doc_tf in body_index.read_posting_list(token):
            if doc_tf[0] not in similarity_dict:
                similarity_dict[doc_tf[0]] = 0
            similarity_dict[doc_tf[0]] += tf_idf_calc(token, doc_tf[0], doc_tf[1])

    for doc in similarity_dict.keys():
        similarity_dict[doc] = similarity_dict[doc]*normalize(tokens)*doc_norm_dict[doc]
        
    top100 = list(sorted(similarity_dict.items(), key=lambda item: item[1], reverse=True)[:100])
    
    for pair in top100:
        res.append((pair[0], doc_title_dict[pair[0]]))

    # END SOLUTION
#     return jsonify(res)
#     return res
    return res




In [21]:
print(search_body())

[(42531713, 'Jonathan Morgan (tennis)'), (68308072, 'Alejandra Pérez Lecaros'), (22291773, 'Daniel Roth'), (47626712, 'Yoel Bouza'), (560176, 'Craig McClanahan'), (28871639, 'Salvatore Falco'), (67611948, 'Jan Laugesen'), (35902457, 'Shree Kitini (10+2) Higher Secondary School'), (59144735, 'Apache Helix'), (52157350, 'Tamara Shukakidze'), (44295156, 'Stacy Christakakis'), (68338141, 'Elisa Giustinianovich'), (57095061, 'Connectifier'), (37784208, 'Steven Takiff'), (22674192, 'Michael Theodoulou'), (64418727, 'Knowledge Graph (disambiguation)'), (36703543, 'Savio Barnes'), (42738347, 'Mohabir Anil Nandlall'), (50070773, 'Timothy Jurka'), (35934951, 'SocialIQ'), (9516775, 'Charissa Tansomboon'), (67910832, 'Tomer Eiges'), (49277625, 'Ahillen Beadle'), (12249721, 'Mark Hoplamazian'), (63641225, 'Ryan Roslansky'), (49106425, 'Peter Skomoroch'), (29720689, 'Stanley B. Lippman'), (43652569, 'Brandi Hitt'), (38613414, 'Bright.com'), (60996273, 'Ibrahim Badr'), (45248740, 'RenewableUK Cymru')

In [13]:
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.bin
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip

--2023-01-11 11:41:58--  https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10356881291 (9.6G) [application/zip]
Saving to: ‘wiki.en.zip’

wiki.en.zip         100%[===================>]   9.65G  38.2MB/s    in 4m 10s  

2023-01-11 11:46:08 (39.5 MB/s) - ‘wiki.en.zip’ saved [10356881291/10356881291]



In [2]:
%cd /home/dataproc

/home/dataproc


In [70]:
page_views_avg = sum(page_views_dict.values())/len(page_views_dict.values())
print (page_views_avg)
page_rank_avg = sum(page_rank_dict.values())/len(page_rank_dict.values())
print(page_rank_avg)

674.0488855666746
1.0000000000249587


In [143]:
def bm25_update(token, doc_id, tf):
    k1 = 1.5
    B = 0.75
    N = 6348910
    avgl = 319.5242353411845
    idf = math.log(((N - body_index.df[token] + 0.5)/(body_index.df[token]+0.5)) + 1, math.e)
    score = (idf * ((tf * (k1 + 1)) / (tf + k1 * (1 - B + B * (doc_len_dict[doc_id] / avgl)))))
    return score

def search(q, weights):
    ''' Returns up to a 100 of your best search results for the query. This is 
        the place to put forward your best search engine, and you are free to
        implement the retrieval whoever you'd like within the bound of the 
        project requirements (efficiency, quality, etc.). That means it is up to
        you to decide on whether to use stemming, remove stopwords, use 
        PageRank, query expansion, etc.

        To issue a query navigate to a URL like:
         http://YOUR_SERVER_DOMAIN/search?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of up to 100 search results, ordered from best to worst where each 
        element is a tuple (wiki_id, title).
    '''
    res = []
#     query = request.args.get('query', '')
#     query = 'best marvel movie'
    query = q
    if len(query) == 0:
        return jsonify(res)
    # BEGIN SOLUTION

    similarity_dict = {}

    tokens = tokenize(query)
    # word2vec
    # Constants
    BODY_WEIGHT, ANCHOR_WEIGHT, TITLE_WEIGHT, PAGE_RANK_WEIGHT, PAGE_VIEW_WEIGHT = weights
#     print(BODY_WEIGHT)
#     print(ANCHOR_WEIGHT)
#     print(TITLE_WEIGHT)


#     BODY_WEIGHT = 0.5
#     ANCHOR_WEIGHT = 2
#     TITLE_WEIGHT = 3
#     PAGE_RANK_WEIGHT = 0.001
#     PAGE_VIEW_WEIGHT = 0.35
    token_doc_titles_occurrences = Counter()
    token_doc_anchor_occurrences = Counter()

    for token in tokens:
        # Searching the term in anchor index
        for doc_tf in anchor_index.read_posting_list(token):
            token_doc_anchor_occurrences[doc_tf[0]] = 1 + token_doc_anchor_occurrences.get(doc_tf[0], 0)

        # Searching the term in title index
        for doc_tf in title_index.read_posting_list(token):
            token_doc_titles_occurrences[doc_tf[0]] = 1 + token_doc_titles_occurrences.get(doc_tf[0], 0)

        # Searching the term in body index
        for doc_tf in body_index.read_posting_list(token):
            if doc_tf[0] not in similarity_dict.keys():
                similarity_dict[doc_tf[0]] = 0

            a = token_doc_anchor_occurrences.get(doc_tf[0], 0) * ANCHOR_WEIGHT
            b = token_doc_titles_occurrences.get(doc_tf[0], 0) * TITLE_WEIGHT
            c = bm25_update(token, doc_tf[0], doc_tf[1]) * BODY_WEIGHT
            d = math.log((page_rank_dict.get(doc_tf[0], 0.00001)/page_rank_avg),10) * PAGE_RANK_WEIGHT
            e = math.log((page_views_dict.get(doc_tf[0], 0.00001)/page_views_avg), 10) * PAGE_VIEW_WEIGHT
            
#             if doc_tf[0] in [21148, 5043544, 32070]:
#                 print('a: '+ str(a/ANCHOR_WEIGHT))
#                 print('b: '+ str(b/TITLE_WEIGHT))
#                 print('c: '+ str(c/BODY_WEIGHT))
#                 print('d: '+ str(d/PAGE_RANK_WEIGHT))
            update =  a + b + c + d + e
#             update =  a + b + c + d 
#             update =  d + b + c  
            similarity_dict[doc_tf[0]] += update


    top100 = list(sorted(similarity_dict.items(), key=lambda item: item[1], reverse=True)[:100])

    for pair in top100:
        res.append((pair[0], doc_title_dict[pair[0]]))

    # END SOLUTION
#     return jsonify(res)
#     return top100
    return res


In [147]:
q = "ship"
print(search(q, weights))

[(268524, 'Ship of Theseus'), (855210, 'Lead ship'), (164340, 'Ship prefix'), (60925, 'Ship commissioning'), (4115690, 'Khufu ship'), (4218865, 'Oasis-class cruise ship'), (35013518, 'The captain goes down with the ship'), (291618, "Her Majesty's Ship"), (187604, 'Vasa (ship)'), (1138566, 'Full-rigged ship'), (1061074, 'Oseberg Ship'), (102848, 'White Ship'), (1354688, 'Ghost Ship (2002 film)'), (17020474, 'List of ship types'), (35964309, 'List of ship directions'), (1128147, 'Turtle ship'), (3561821, 'Sister ship'), (93099, 'Ship of the line'), (146351, 'Hospital ship'), (875717, 'Merchant ship'), (723505, 'Gokstad ship'), (786056, 'Cargo ship'), (198016, 'Museum ship'), (407592, 'Hulk (ship type)'), (1037955, 'Ship breaking'), (633917, 'Ghost ship'), (3249592, 'Auxiliary ship'), (198201, 'Liberty ship'), (31186721, 'Quantum-class cruise ship'), (2724407, 'Rotor ship'), (17160742, 'Displacement (ship)'), (2293646, 'Endurance (1912 ship)'), (198199, 'Victory ship'), (491457, 'United S

In [122]:
# weights = [0.75133306, 0.08561443, 0.74563351, 0.97631322, 0.54191462]
# print(search('Rick and Morty' ,weights))

In [141]:
print(page_rank_dict[68380237])

0.1501208493870428


In [123]:
# print(queries['Winter'])

In [111]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list): 
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
#     print("recall: " + len(precisions)+" out of 40")
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [130]:
print(queries["Winter"])

[1511596, 52709838, 1673945, 17349106, 8521120, 3060382, 16615604, 8438818, 8351234, 979072, 64928991, 36439749, 1817908, 1971153, 30276826, 4538366, 1298502, 22933429, 6511088, 38950, 19431459, 1221144, 19938267, 1843684, 34069, 6201653, 33672235, 3548574, 1372169, 65601132, 38416091, 1088531, 2020857, 316711, 34061, 9825536, 22190045, 43343961, 1632099, 962053, 961505]


In [140]:
def get_ap(q_title, weights_list):
    res = search(q_title ,weights)
    predicted, _ = zip(*res)
    actual = queries[q_title]
    ap = average_precision(actual, predicted)
    return ap

q_title = "World Cup 2022"
# weights = [0.75133306, 0.08561443, 0.74563351, 0.97631322, 0.54191462]
# Winter query weights: 
weights = [0.55754871, 1.25785412, 0.54434859, 0.09795087, 0.38754463]
print(get_ap(q_title, weights))
# actual = [54046846, 49170369, 55339286, 62417830, 54251265, 41283158, 65819511, 49029294, 67520032, 49134382, 42311608, 63656330, 52261594, 41185040, 43794572, 64413225, 49131135, 49127974, 43794574, 63656361, 49128142, 41699729, 51759111, 47762921, 55708102, 63656365, 55339303, 57390230, 67830379, 61805032, 26091326, 54802759]
# weights = [0.75133306, 0.08561443, 0.74563351, 0.97631322, 0.54191462]
# res = search('Rick and Morty' ,weights)
# predicted, _ = zip(*res)
# ap = average_precision(actual, predicted)
# print(ap)

0.652


In [63]:
# def queries_ap(weights, q_num=1):
#     queries = list(queries.items())[:q_num]
#     qs_res = []

#     for q, true_wids in queries:
#         duration, ap = None, None
#         t_start = time()
#         pred_wids = search(q, weights)
#         duration = time() - t_start
#         ap = average_precision(true_wids, pred_wids)
#         qs_res.append((q, duration, ap, weights))
#     return qs_res

In [108]:
def queries_ap(weights, query):
#     queries = list(queries.items())[:q_num]
#     qs_res = []
    queries = [query]
    for q, true_wids in queries:
        ap = None
#         -----------------------------------------------------
        res = search(q, weights)
        pred_wids, _ = zip(*res)
        ap = average_precision(true_wids, pred_wids)
#         qs_res.append((q, duration, ap, weights))
    return ap

In [113]:

# query_to_run = (q_title, queries[q_title])
# print(query_to_run)

('Winter', [1511596, 52709838, 1673945, 17349106, 8521120, 3060382, 16615604, 8438818, 8351234, 979072, 64928991, 36439749, 1817908, 1971153, 30276826, 4538366, 1298502, 22933429, 6511088, 38950, 19431459, 1221144, 19938267, 1843684, 34069, 6201653, 33672235, 3548574, 1372169, 65601132, 38416091, 1088531, 2020857, 316711, 34061, 9825536, 22190045, 43343961, 1632099, 962053, 961505])


In [119]:
import numpy as np
from scipy.optimize import minimize


def print_optimized_weights(q_title):
    query_to_run = (q_title, queries[q_title])
#     print(query_to_run)
    # Define the function to be maximized
    def y(x):
        x1, x2, x3, x4, x5 = x
        weights = x.tolist()
        res =  queries_ap(weights,query_to_run)
        return -1 * res

    # Define the initial point
    x0 = np.random.uniform(0, 1, 5)

    # Number of iterations
    num_iterations = 10

    # list to hold the best points
    best_points = []

    # Perform the optimization num_iterations times
    for i in range(num_iterations):
        # Optimize the function
        bounds = [(0, None)]*5

        result = minimize(y, x0, bounds=bounds, method='L-BFGS-B')
        # add the point to the best points list
        best_points.append((-result.fun, result.x))
        x0 = result.x
        # Randomize x0 a little bit to avoid getting stuck in a local minimum
        x0 += np.random.normal(scale=0.1, size=5)
#         print(x0)
    # Sort the list of best points and take the top 10
    best_points.sort(key= lambda y_x: y_x[0] ,reverse=True)
    top_10 = best_points[:10]

    # Print the 10 best points
    print("anchor, title, body, page_rank, page_views")
    for i, point in enumerate(top_10):
        print("{}. y = {}, x = {}".format(i+1, point[0], point[1]))


In [118]:
q_title = "Rick and Morty"
print_optimized_weights(q_title)

[0.72003659 0.62859251 0.41320103 0.64153128 0.45821703]
[0.70467526 0.67284511 0.50217344 0.63171468 0.40092812]
[0.73573968 0.73870666 0.52359982 0.61458126 0.48539368]
[0.53134471 0.84669479 0.49451533 0.46530829 0.5732673 ]
[0.6462671  0.75457893 0.61579258 0.64595072 0.55396991]
[0.69688303 0.6558302  0.57247377 0.58757528 0.73187791]
[0.70745614 0.70524924 0.64024859 0.53248478 0.7185971 ]
[0.77795504 0.66104408 0.76544221 0.65682795 0.74708016]
[0.66565863 0.71939317 0.85524436 0.6167213  0.74657466]
[0.5662838  0.67381974 0.78139254 0.5649084  0.72723188]
anchor, title, body, page_rank, page_views
1. y = 0.818, x = [0.66565863 0.71939317 0.85524436 0.6167213  0.74657466]
2. y = 0.816, x = [0.77795504 0.66104408 0.76544221 0.65682795 0.74708016]
3. y = 0.813, x = [0.5662838  0.67381974 0.78139254 0.5649084  0.72723188]
4. y = 0.81, x = [0.70745614 0.70524924 0.64024859 0.53248478 0.7185971 ]
5. y = 0.807, x = [0.6462671  0.75457893 0.61579258 0.64595072 0.55396991]
6. y = 0.802,

In [121]:
q_title = "Winter"
print_optimized_weights(q_title)

anchor, title, body, page_rank, page_views
1. y = 0.751, x = [ 0.3802622   1.15174287  0.66534942  0.04572521 -0.02795971]
2. y = 0.736, x = [0.55754871 1.25785412 0.54434859 0.09795087 0.38754463]
3. y = 0.718, x = [ 0.70739086  1.01978448  0.34099951 -0.09697754  0.25905376]
4. y = 0.712, x = [ 0.47024579  1.0151511   0.31649317 -0.00362412  0.52768426]
5. y = 0.63, x = [0.50676697 1.00784559 0.3411947  0.0340439  0.65066275]
6. y = 0.618, x = [0.44357598 1.0101824  0.14261672 0.0160672  0.88657244]
7. y = 0.508, x = [0.5154044  1.01465605 0.57672439 0.05566338 0.09372452]
8. y = 0.507, x = [ 0.52229925  0.92198258 -0.05646227  0.1800847   0.84241589]
9. y = 0.491, x = [0.52162412 1.02846453 0.07928739 0.06320404 0.73917427]
10. y = 0.373, x = [0.52621377 1.17490465 0.58755963 0.07533695 0.09622741]


In [ ]:
# page_rank_max =